In [42]:
import psycopg2

In [43]:
connect = psycopg2.connect(dbname="fnf", host="prd-dt-redshift.conhugwtudej.ap-northeast-2.redshift.amazonaws.com", port=5439, user="wecode", password="Wecode@2022!#")

In [44]:
cursor = connect.cursor()

In [45]:
brand = 'M'
season = "'22S'"
season_py = "'21S'"
season_py2 = "'20S'"
category = '맨투맨'
sub_category = "'후드','맨투맨'"
adult_kids = '성인'
start_dt = '2021-12-28'
end_dt = '2022-02-06'
end_dt_this_week = '2022-02-06'

In [46]:
query = """
WITH int_stock AS (
    SELECT '당해'         term_cls
         , stock_qty AS int_stock_qty
    FROM prcs.db_scs_w a,
         prcs.db_prdt b
    WHERE a.prdt_cd = b.prdt_cd
      AND a.brd_cd = '{para_brand}'
      AND a.sesn in ({para_season}) --당해
      AND cat_nm = '{para_category}'
      AND sub_cat_nm in ({para_sub_category})
      AND adult_kids_nm = '{para_adult_kids}'
      AND end_dt = '{para_end_dt_this_week}' - 7
    UNION ALL
    SELECT '전년'         term_cls
         , stock_qty as int_stock_qty
    FROM prcs.db_scs_w a,
         prcs.db_prdt b
    WHERE a.prdt_cd = b.prdt_cd
      AND a.brd_cd = '{para_brand}'
      AND a.sesn in ({para_season_py}) --전년
      AND cat_nm = '{para_category}'
      AND sub_cat_nm in ({para_sub_category})
      AND adult_kids_nm = '{para_adult_kids}'
      AND end_dt = '{para_end_dt_this_week}' - 364 - 7
),
     term_sales AS (
         SELECT '당해'       term_cls
              , stor_qty_kor                        AS stor_qty_kor_term
              , sale_nml_qty_cns + sale_ret_qty_cns AS sale_qty_kor_term
              , sale_nml_qty_cns + sale_ret_qty_cns AS sale_qty_w
              , stock_qty                           AS stock_kor

         FROM prcs.db_scs_w a,
              prcs.db_prdt b
         WHERE a.prdt_cd = b.prdt_cd
           AND a.brd_cd = '{para_brand}'
           AND a.sesn in ({para_season}) --당해
           AND cat_nm = '{para_category}'
           AND sub_cat_nm in ({para_sub_category})
           AND adult_kids_nm = '{para_adult_kids}'
           AND end_dt = '{para_end_dt_this_week}'
         UNION ALL
         SELECT '전년'       term_cls
              , stor_qty_kor                        AS stor_qty_kor_term
              , sale_nml_qty_cns + sale_ret_qty_cns AS sale_qty_kor_term
              , sale_nml_qty_cns + sale_ret_qty_cns AS  sale_qty_w
              , stock_qty                           AS stock_kor
         FROM prcs.db_scs_w a,
              prcs.db_prdt b
         WHERE a.prdt_cd = b.prdt_cd
           AND a.brd_cd = '{para_brand}'
           AND a.sesn in ({para_season_py}) --전년
           AND cat_nm = '{para_category}'
           AND sub_cat_nm in ({para_sub_category})
           AND adult_kids_nm = '{para_adult_kids}'
           AND end_dt = '{para_end_dt_this_week}' - 364
     ),

     sale_4wk AS (
         SELECT '당해'       term_cls
              , sale_nml_qty_cns + sale_ret_qty_cns AS sale_qty_kor_4wk
         FROM prcs.db_scs_w a,
              prcs.db_prdt b
         WHERE a.prdt_cd = b.prdt_cd
           AND a.brd_cd = '{para_brand}'
           AND a.sesn in ({para_season}) --당해
           AND cat_nm = '{para_category}'
           AND sub_cat_nm in ({para_sub_category})
           AND adult_kids_nm = '{para_adult_kids}'
           AND end_dt between '{para_end_dt_this_week}' - 7 * 3 AND '{para_end_dt_this_week}'
         UNION ALL
         SELECT '전년'       term_cls
              , sale_nml_qty_cns + sale_ret_qty_cns AS sale_qty_kor_4wk
         FROM prcs.db_scs_w a,
              prcs.db_prdt b
         WHERE a.prdt_cd = b.prdt_cd
           AND a.brd_cd = '{para_brand}'
           AND a.sesn in ({para_season_py}) --전년
           AND cat_nm = '{para_category}'
           AND sub_cat_nm in ({para_sub_category})
           AND adult_kids_nm = '{para_adult_kids}'
           AND end_dt between '{para_end_dt_this_week}' - 364 - 7 * 3 AND '{para_end_dt_this_week}' - 364
     )
SELECT term_cls
     , int_stock_qty
     , stor_qty_kor_term
     , sale_qty_kor_term
     , sale_qty_w
     , avg_4wk_sale_qty
     , stock_kor
     , case when sale_qty_w = 0 then 0 else ROUND(stock_kor::numeric / sale_qty_w) end           AS woi
     , case when avg_4wk_sale_qty=0 then 0 else ROUND(stock_kor::numeric / avg_4wk_sale_qty) end AS woi_4wks
FROM (
         SELECT term_cls
              , SUM(int_stock_qty)                  int_stock_qty
              , SUM(stor_qty_kor_term)              stor_qty_kor_term
              , SUM(sale_qty_kor_term)              sale_qty_kor_term
              , SUM(sale_qty_w)                     sale_qty_w
              , SUM(stock_kor)                      stock_kor
              , ROUND(SUM(sale_qty_kor_4wk) / 4) AS avg_4wk_sale_qty
         FROM (
                  SELECT term_cls
                       , 0 AS int_stock_qty
                       , 0 AS stor_qty_kor_term
                       , 0 AS sale_qty_kor_term
                       , 0 AS sale_qty_w
                       , 0 AS stock_kor
                       , sale_qty_kor_4wk
                  FROM sale_4wk
                  UNION ALL
                  SELECT term_cls
                       , 0 AS int_stock_qty
                       , stor_qty_kor_term
                       , sale_qty_kor_term
                       , sale_qty_w
                       , stock_kor
                       , 0 AS sale_qty_kor_4wk
                  FROM term_sales
                  UNION ALL
                  SELECT term_cls
                       , int_stock_qty
                       , 0 AS stor_qty_kor_term
                       , 0 AS sale_qty_kor_term
                       , 0 AS sale_qty_w
                       , 0 AS stock_kor
                       , 0 AS sale_qty_kor_4wk
                  FROM int_stock
              ) a
         GROUP BY term_cls
     ) a
ORDER BY term_cls

        """.format(
            para_brand=brand,
            para_season=season,
            para_season_py=season_py,
            para_season_py2=season_py2,
            para_category=category,
            para_sub_category=sub_category,
            para_adult_kids=adult_kids,
            para_start_dt=start_dt,
            para_end_dt=end_dt,
            para_end_dt_this_week=end_dt_this_week,
        )

In [47]:
cursor.execute(query)

In [9]:
cursor.execute("ROLLBACK")
connect.commit()

In [48]:
query_output = cursor.fetchall()

In [49]:
query_output

[('당해', 152254, 8380, 7257, 7257, 7961.0, 152673, Decimal('21'), 19.0),
 ('전년', 72419, 12391, 11616, 11616, 11393.0, 73050, Decimal('6'), 6.0)]

In [50]:
from utils.redshift_data import RedshiftData

In [51]:
redshift_data = RedshiftData(connect, query)


In [52]:
data = redshift_data.get_data()

/opt/homebrew/Caskroom/miniconda/base/envs/fnf/lib/python3.8/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [54]:
data

,term_cls,int_stock_qty,stor_qty_kor_term,sale_qty_kor_term,sale_qty_w,avg_4wk_sale_qty,stock_kor,woi,woi_4wks
0,당해,152254,8380,7257,7257,7961.0,152673,21.0,19.0
1,전년,72419,12391,11616,11616,11393.0,73050,6.0,6.0


In [38]:
import pandas.io.sql as psql

In [40]:
psql.read_sql(query,connect)

/opt/homebrew/Caskroom/miniconda/base/envs/fnf/lib/python3.8/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,term_cls,int_stock_qty,stor_qty_kor_term,sale_qty_kor_term,sale_qty_w,avg_4wk_sale_qty,stock_kor,woi,woi_4wks
0,당해,152254,8380,7257,7257,7961.0,152673,21.0,19.0
1,전년,72419,12391,11616,11616,11393.0,73050,6.0,6.0


In [59]:
data

,term_cls,int_stock_qty,stor_qty_kor_term,sale_qty_kor_term,sale_qty_w,avg_4wk_sale_qty,stock_kor,woi,woi_4wks
0,당해,152254,8380,7257,7257,7961.0,152673,21.0,19.0
1,전년,72419,12391,11616,11616,11393.0,73050,6.0,6.0


In [77]:
result = [{
    "term_cls": item["term_cls"],                       #해당년도
    "int_stock_qty": item["int_stock_qty"],             #기초재고
    "stor_qty_kor_term": item["stor_qty_kor_term"],     #주간입고 
    "sale_qty_w	": item["sale_qty_w"],                  #주간판매
    "avg_4wk_sale_qty": item["avg_4wk_sale_qty"],       #4주평균
    "stock_kor": item["stock_kor"],                     #기말재고
    "woi_4wks": item["woi_4wks"],                       #재고주수:4주평균     
    }for __, item in data.iterrows()
]

In [78]:
result

[{'term_cls': '당해',
  'int_stock_qty': 152254,
  'stor_qty_kor_term': 8380,
  'sale_qty_w\t': 7257,
  'avg_4wk_sale_qty': 7961.0,
  'stock_kor': 152673,
  'woi_4wks': 19.0},
 {'term_cls': '전년',
  'int_stock_qty': 72419,
  'stor_qty_kor_term': 12391,
  'sale_qty_w\t': 11616,
  'avg_4wk_sale_qty': 11393.0,
  'stock_kor': 73050,
  'woi_4wks': 6.0}]

In [79]:
result[0]

{'term_cls': '당해',
 'int_stock_qty': 152254,
 'stor_qty_kor_term': 8380,
 'sale_qty_w\t': 7257,
 'avg_4wk_sale_qty': 7961.0,
 'stock_kor': 152673,
 'woi_4wks': 19.0}

In [80]:
result[1]['stock_kor']

73050